In [1]:
!pip install torchmetrics

!pip install tabulate

In [1]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification, GPT2Config
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np

from tabulate import tabulate
from tqdm import trange
import random
from torchmetrics.classification import Recall, Accuracy, AUROC, Precision

/home/ss6928/.conda/envs/myenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
FILL = "FILL"

In [4]:
!wget 'https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip'
!unzip -o smsspamcollection.zip

--2024-05-04 18:04:53--  https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip
Resolving gw-srv-01.rc.zi.columbia.edu (gw-srv-01.rc.zi.columbia.edu)... 10.198.22.10
Connecting to gw-srv-01.rc.zi.columbia.edu (gw-srv-01.rc.zi.columbia.edu)|10.198.22.10|:3128... connected.
Proxy request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘smsspamcollection.zip.2’

    [  <=>                                  ] 203,415      755KB/s   in 0.3s   

2024-05-04 18:04:54 (755 KB/s) - ‘smsspamcollection.zip.2’ saved [203415]

Archive:  smsspamcollection.zip
  inflating: SMSSpamCollection       
  inflating: readme                  


In [5]:
!unzip -o smsspamcollection.zip

Archive:  smsspamcollection.zip
  inflating: SMSSpamCollection       
  inflating: readme                  


In [6]:
!head -10 SMSSpamCollection

ham	Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...
ham	Ok lar... Joking wif u oni...
spam	Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's
ham	U dun say so early hor... U c already then say...
ham	Nah I don't think he goes to usf, he lives around here though
spam	FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, £1.50 to rcv
ham	Even my brother is not like to speak with me. They treat me like aids patent.
ham	As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your callertune for all Callers. Press *9 to copy your friends Callertune
spam	WINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.
spam	H

In [7]:
!wget 'https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip'
!unzip -o smsspamcollection.zip

--2024-05-04 18:04:55--  https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip
Resolving gw-srv-01.rc.zi.columbia.edu (gw-srv-01.rc.zi.columbia.edu)... 10.198.22.10
Connecting to gw-srv-01.rc.zi.columbia.edu (gw-srv-01.rc.zi.columbia.edu)|10.198.22.10|:3128... connected.
Proxy request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘smsspamcollection.zip.3’

    [  <=>                                  ] 203,415      752KB/s   in 0.3s   

2024-05-04 18:04:55 (752 KB/s) - ‘smsspamcollection.zip.3’ saved [203415]

Archive:  smsspamcollection.zip
  inflating: SMSSpamCollection       
  inflating: readme                  


In [3]:
file_path = 'SMSSpamCollection'
df = pd.DataFrame({'label':int(), 'text':str()}, index = [])
with open(file_path) as f:
    for line in f.readlines():
        split = line.split('\t')
        df = pd.concat([
                df,
                pd.DataFrame.from_dict({
                    'label': [1 if split[0] == 'spam' else 0],
                    'text': [split[1]]
                })
            ],
            ignore_index=True
        )
df.head()

,label,text
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...\n
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
text = df.text.values
labels = df.label.values

In [5]:
# Set to the GPT2Tokenizer and set lower case to True
tokenizer = GPT2Tokenizer.from_pretrained("gpt2", add_prefix_space=True)
tokenizer.pad_token = tokenizer.eos_token  
# Set the padding to 'left' or 'right'? left
# Remember we want to use the last token's embedding to represent the entire sentence
tokenizer.padding_side = 'left'

In [11]:
def print_rand_sentence():
    '''Displays the tokens and respective IDs of a random text sample'''
    index = random.randint(0, len(text)-1)
    print(text[index])
    table = np.array([tokenizer.tokenize(text[index]),
                    tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text[index]))]).T
    print(tabulate(table,
                 headers = ['Tokens', 'Token IDs'],
                 tablefmt = 'fancy_grid'))

print_rand_sentence()

Double eviction this week - Spiral and Michael and good riddance to them!

╒═══════════╤═════════════╕
│ Tokens    │   Token IDs │
╞═══════════╪═════════════╡
│ ĠDouble   │       11198 │
├───────────┼─────────────┤
│ Ġeviction │       38738 │
├───────────┼─────────────┤
│ Ġthis     │         428 │
├───────────┼─────────────┤
│ Ġweek     │        1285 │
├───────────┼─────────────┤
│ Ġ-        │         532 │
├───────────┼─────────────┤
│ ĠSpiral   │       46727 │
├───────────┼─────────────┤
│ Ġand      │         290 │
├───────────┼─────────────┤
│ ĠMichael  │        3899 │
├───────────┼─────────────┤
│ Ġand      │         290 │
├───────────┼─────────────┤
│ Ġgood     │         922 │
├───────────┼─────────────┤
│ Ġr        │         374 │
├───────────┼─────────────┤
│ idd       │        1638 │
├───────────┼─────────────┤
│ ance      │         590 │
├───────────┼─────────────┤
│ Ġto       │         284 │
├───────────┼─────────────┤
│ Ġthem     │         606 │
├───────────┼─────────────┤
│

In [6]:
token_id = []
attention_masks = []

def preprocessing(input_text, tokenizer):
    '''
  Returns <class transformers.tokenization_utils_base.BatchEncoding> with the following fields:
    - input_ids: list of token ids
    - token_type_ids: list of token type ids
    - attention_mask: list of indices (0,1) specifying which tokens should considered by the model (return_attention_mask = True).
      '''
  # Use the tokenizer and the encode_plus methods to return the right data we'll need
  # Set max_length = 32 and return_tokens = 'pt'
  # Set other fields to the appropriate booleans needed
    return tokenizer.encode_plus(
        text=input_text,
        add_special_tokens=True,
        max_length=32,
        return_attention_mask=True,
        padding='max_length',
        truncation=True,
        return_tensors='pt'  # Return PyTorch tensors
    )


for sample in text:
    encoding_dict = preprocessing(sample, tokenizer)
    token_id.append(encoding_dict['input_ids'])
    attention_masks.append(encoding_dict['attention_mask'])


# Gather all the torch_id, attention masks, and labels
token_id = torch.cat(token_id, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

In [53]:
def print_rand_sentence_encoding():
    '''Displays tokens, token IDs and attention mask of a random text sample'''
    index = random.randint(0, len(text) - 1)
    tokens = tokenizer.tokenize(tokenizer.decode(token_id[index]))[1:]
    print(tokens)
    token_ids = [i.numpy() for i in token_id[index]]
    attention = [i.numpy() for i in attention_masks[index]]
    table = np.array([tokens, token_ids, attention]).T
    print(
        tabulate(
            table,
            headers = ['Tokens', 'Token IDs', 'Attention Mask'],
            tablefmt = 'fancy_grid')
    )

print_rand_sentence_encoding()

['<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', 'ĠWell', 'Ġimm', 'a', 'Ġdefinitely', 'Ġneed', 'Ġto', 'Ġrest', 'ock', 'Ġbefore', 'Ġthanks', 'giving', ',', 'ĠI', "'ll", 'Ġlet', 'Ġyou', 'Ġknow', 'Ġwhen', 'ĠI', "'m", 'Ġout', 'Ċ']
╒═══════════════╤═════════════╤══════════════════╕
│ Tokens        │   Token IDs │   Attention Mask │
╞═══════════════╪═════════════╪══════════════════╡
│ <|endoftext|> │       50256 │                0 │
├───────────────┼─────────────┼──────────────────┤
│ <|endoftext|> │       50256 │                0 │
├───────────────┼─────────────┼──────────────────┤
│ <|endoftext|> │       50256 │                0 │
├───────────────┼─────────────┼──────────────────┤
│ <|endoftext|> │       50256 │                0 │
├───────────────┼─────────────┼──────────────────┤
│ <|endoftext|> │       50256 │                0 │
├───────────────┼─────────────┼────────

In [7]:
val_ratio = 0.2
# Recommended batch size: 16, 32. See: https://arxiv.org/pdf/1810.04805.pdf
batch_size = 16

# Indices of the train and validation splits stratified by labels
# Use train_test_split
train_idx, val_idx = train_test_split(
    range(len(labels)),  # indices to split
    test_size=val_ratio,  # validation set size
    stratify=labels,      # stratify by labels to ensure balanced classes
    random_state=42       # seed for reproducibility
)

# Train and validation sets
# Set to TensorDataset
train_set = TensorDataset(token_id[train_idx], attention_masks[train_idx], labels[train_idx])
val_set = TensorDataset(token_id[val_idx], attention_masks[val_idx], labels[val_idx])
# Prepare DataLoader

train_dataloader = DataLoader(train_set, batch_size=16, shuffle=True)
validation_dataloader = DataLoader(val_set, batch_size=16, shuffle=False)

### Load specific versions of the model

In [14]:
# Load the BertForSequenceClassification model
# Do not ouput the attentions and all hidden states

config = GPT2Config.from_pretrained('gpt2',
                                    output_attentions=False,
                                    output_hidden_states=False)


# Set to 'gpt2' (the smallest GPT2 which is 120 M parameters)
# Use the config above and set other labels as needed
model = GPT2ForSequenceClassification.from_pretrained('gpt2', config=config)

# Set the pad token id to the eos token id
model.config.pad_token_id = model.config.eos_token_id

# Recommended learning rates (Adam): 5e-5, 3e-5, 2e-5
# See: https://arxiv.org/pdf/1810.04805.pdf
optimizer = torch.optim.AdamW(
    model.parameters(),
    lr = 5e-5,
    eps = 1e-08
)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Set the model to the right device

In [15]:
# If on GPU, do as below
device = 'cuda'

In [59]:
model = model.to(device)

# Recommended number of epochs: See: https://arxiv.org/pdf/1810.04805.pdf
epochs = 2

In [61]:
# Print all the layers of this GPT2 model and the number of parameters per layer
# If this is False, fine tune just the classifier layer and leave all other GPT2 parameters alone
# If this is True, fine tune everything
fine_tune = True
total_parameters = 0

# Print layers and count parameters
for name, param in model.named_parameters():
    # Get the number of parameters, shape, and gradient requirement
    param_count = param.numel()
    total_parameters += param_count
    print(f"{name}: {param_count} {param.shape} {param.requires_grad}")

# Check total parameters
print(f"Total parameters in model: {total_parameters}")
# Turn off gradients using the above
if not fine_tune:
    # Turn off gradients for all parameters except the classification layer
    for name, param in model.named_parameters():
        if 'classifier' not in name:  
            param.requires_grad = False
    
assert(total_parameters == 124441344)

transformer.wte.weight: 38597376 torch.Size([50257, 768]) True
transformer.wpe.weight: 786432 torch.Size([1024, 768]) True
transformer.h.0.ln_1.weight: 768 torch.Size([768]) True
transformer.h.0.ln_1.bias: 768 torch.Size([768]) True
transformer.h.0.attn.c_attn.weight: 1769472 torch.Size([768, 2304]) True
transformer.h.0.attn.c_attn.bias: 2304 torch.Size([2304]) True
transformer.h.0.attn.c_proj.weight: 589824 torch.Size([768, 768]) True
transformer.h.0.attn.c_proj.bias: 768 torch.Size([768]) True
transformer.h.0.ln_2.weight: 768 torch.Size([768]) True
transformer.h.0.ln_2.bias: 768 torch.Size([768]) True
transformer.h.0.mlp.c_fc.weight: 2359296 torch.Size([768, 3072]) True
transformer.h.0.mlp.c_fc.bias: 3072 torch.Size([3072]) True
transformer.h.0.mlp.c_proj.weight: 2359296 torch.Size([3072, 768]) True
transformer.h.0.mlp.c_proj.bias: 768 torch.Size([768]) True
transformer.h.1.ln_1.weight: 768 torch.Size([768]) True
transformer.h.1.ln_1.bias: 768 torch.Size([768]) True
transformer.h.1.a

### Train the model

In [65]:
# Use torchmetrics to set up accuracy, recall, precision, and auroc
accuracy = Accuracy(task='binary')
recall = Recall(task='binary')
precision = Precision(task='binary')
auroc = AUROC(task='binary',num_classes=2)    

In [ ]:
# Main training / validation loop
for _ in trange(epochs, desc = 'Epoch'):

    # ========== Training ==========

    # Set model to training mode
    model.train()

    # Tracking variables
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0

    for step, batch in enumerate(train_dataloader):
        # Put each element of batch onto the device
        batch = tuple(t.to(device) for t in batch)  

        # Unpack the batch
        b_input_ids, b_input_mask, b_labels = batch

        # Set gradients to zero
        optimizer.zero_grad()

        # Forward pass
        train_output = model(b_input_ids, attention_mask=b_input_mask, labels=b_labels)

        # Backward pass
        loss = train_output.loss
        loss.backward()
        optimizer.step()
        
        # Update tracking variables
        tr_loss += loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1

    # ========== Validation ==========

    # Set model to evaluation mode
    model.eval()

    # Tracking variables
    val_accuracy = []
    val_precision = []
    val_recall = []
    val_auroc = []

    for batch in validation_dataloader:
        batch = tuple(t.to(device) for t in batch)

        # Unpack the batch
        b_input_ids, b_input_mask, b_labels = batch

        with torch.no_grad():
          # Forward pass
            eval_output = model(b_input_ids, attention_mask=b_input_mask)


        # Calculate validation metrics
        labels = b_labels
        predicted_labels = torch.argmax(outputs.logits, dim=1)

        val_accuracy.append(accuracy(predicted_labels, labels).item())
        val_recall.append(recall(predicted_labels, labels).item())
        val_precision.append(precision(predicted_labels, labels).item())
        val_auroc.append(auroc(outputs.logits, labels).item())
        
    print('\n\t - Train loss: {:.4f}'.format(tr_loss / nb_tr_steps))
    print('\t - Validation Accuracy: {:.4f}'.format(sum(val_accuracy)/len(val_accuracy)))
    print('\t - Validation Precision: {:.4f}'.format(sum(val_precision)/len(val_precision)))
    print('\t - Validation Recall: {:.4f}'.format(sum(val_recall)/len(val_recall)))
    print('\t - Validation AUROC: {:.4f}\n'.format(sum(val_auroc)/len(val_auroc)))

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torchmetrics/utilities/prints.py:43: UserWarning: No positive samples in targets, true positive value should be meaningless. Returning zero tensor in true positive score
  warnings.warn(*args, **kwargs)  # noqa: B028
Epoch:  50%|█████     | 1/2 [31:09<31:09, 1869.30s/it]


	 - Train loss: 0.1321
	 - Validation Accuracy: 0.9911
	 - Validation Precision: 0.9357
	 - Validation Recall: 0.8964
	 - Validation AUROC: 0.9263



Epoch: 100%|██████████| 2/2 [1:01:45<00:00, 1852.70s/it]


	 - Train loss: 0.0426
	 - Validation Accuracy: 0.9938
	 - Validation Precision: 0.9381
	 - Validation Recall: 0.9310
	 - Validation AUROC: 0.9426



### Test on a specific sentence, see the outcome

In [66]:
new_sentence = 'WINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.'

# We need Token IDs and Attention Mask for inference on the new sentence
test_ids = []
test_attention_mask = []

# Apply the tokenizer
encoding = preprocessing(new_sentence, tokenizer)

# Extract IDs and Attention Mask
test_ids.append(encoding['input_ids'])
test_attention_mask.append(encoding['attention_mask'])
test_ids = torch.cat(test_ids, dim = 0)
test_attention_mask = torch.cat(test_attention_mask, dim = 0)

# Forward pass, calculate logit predictions
with torch.no_grad():
    output = model(test_ids.to(device), token_type_ids = None, attention_mask = test_attention_mask.to(device))

prediction = 'Spam' if np.argmax(output.logits.cpu().numpy()).flatten().item() == 1 else 'Ham'

print('Input Sentence: ', new_sentence)
print('Predicted Class: ', prediction)

Input Sentence:  WINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.
Predicted Class:  Spam


### Questions

Question 1: Run the above by fine tuning GPT2 and the classfier head and by not doing this (using GPT2 as a feature encoder). What is the gap between this? What are the metrics we get in each case?

Solution: FILL Paste your output here

In [71]:
#### no fine-tuning

new_sentence = 'WINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.'

# We need Token IDs and Attention Mask for inference on the new sentence
test_ids = []
test_attention_mask = []

# Apply the tokenizer
encoding = preprocessing(new_sentence, tokenizer)

# Extract IDs and Attention Mask
test_ids.append(encoding['input_ids'])
test_attention_mask.append(encoding['attention_mask'])
test_ids = torch.cat(test_ids, dim = 0)
test_attention_mask = torch.cat(test_attention_mask, dim = 0)

model = GPT2ForSequenceClassification.from_pretrained('gpt2').to('cuda')

# Forward pass, calculate logit predictions
with torch.no_grad():
    output = model(test_ids.to(device), token_type_ids = None, attention_mask = test_attention_mask.to(device))

prediction = 'Spam' if np.argmax(output.logits.cpu().numpy()).flatten().item() == 1 else 'Ham'

print('Input Sentence: ', new_sentence)
print('Predicted Class: ', prediction)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Input Sentence:  WINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.
Predicted Class:  Ham


As we can see, if we used the non fine-tuned model it predicts the wrong class ("HAM") instead of spam.

In [11]:
#### no fine-tuning
config = GPT2Config.from_pretrained('gpt2',
                                    output_attentions=False,
                                    output_hidden_states=False)


# Set to 'gpt2' (the smallest GPT2 which is 120 M parameters)
# Use the config above and set other labels as needed
device = 'cuda'

model = GPT2ForSequenceClassification.from_pretrained('gpt2', config=config).to('cuda')
device = 'cuda'
# Set the pad token id to the eos token id
model.config.pad_token_id = model.config.eos_token_id

# Recommended learning rates (Adam): 5e-5, 3e-5, 2e-5
# See: https://arxiv.org/pdf/1810.04805.pdf
optimizer = torch.optim.AdamW(
    model.parameters(),
    lr = 5e-5,
    eps = 1e-08
)
model.eval()

# Tracking variables
val_accuracy = []
val_precision = []
val_recall = []
val_auroc = []

accuracy = Accuracy(task='binary').to('cuda')
recall = Recall(task='binary').to('cuda')
precision = Precision(task='binary').to('cuda')
auroc = AUROC(task='binary',num_classes=2).to('cuda')


for batch in validation_dataloader:
    batch = tuple(t.to(device) for t in batch)

    # Unpack the batch
    b_input_ids, b_input_mask, b_labels = batch

    with torch.no_grad():
      # Forward pass
        eval_output = model(b_input_ids, attention_mask=b_input_mask)


    # Calculate validation metrics
    labels = b_labels
    predicted_labels = torch.argmax(eval_output.logits, dim=1)

    val_accuracy.append(accuracy(predicted_labels, labels).item())
    val_recall.append(recall(predicted_labels, labels).item())
    val_precision.append(precision(predicted_labels, labels).item())
    #val_auroc.append(auroc(logits, labels).item())

print('\t - Validation Accuracy: {:.4f}'.format(sum(val_accuracy)/len(val_accuracy)))
print('\t - Validation Precision: {:.4f}'.format(sum(val_precision)/len(val_precision)))
print('\t - Validation Recall: {:.4f}'.format(sum(val_recall)/len(val_recall)))

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


	 - Validation Accuracy: 0.4239
	 - Validation Precision: 0.1762
	 - Validation Recall: 0.7967


As we can see, if we used the non fine-tuned model, the Valdidation accuracy, precision and recall are much lower compared to the fine-tuned model